# Jeu de devinette : Père Fouras vs Laurent Jalabert

Dans ce notebook, nous allons créer une interaction entre deux agents : le Père Fouras, qui fait deviner des mots ou expressions en utilisant des charades et réponses énigmatiques, et Laurent Jalabert, qui devine en posant des questions fermées.

La stratégie de sélection est que les agents parlent chacun à leur tour. La partie se termine lorsque Laurent Jalabert devine le mot ou l'expression du Père Fouras.

In [38]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, *-*"
#!import ../Config/Settings.cs

Installed Packages Microsoft.SemanticKernel, 1.13.0 Microsoft.SemanticKernel.Agents.Core, 1.13.0-alpha

In [39]:

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("../config/Settings.json");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

//var kernel = builder.Build();

Définissons maintenant les prompts pour nos agents Père Fouras et Laurent Jalabert, en veillant à ce que le mot à deviner soit inséré dans le prompt système du Père Fouras.

In [40]:
const string pereFourasSystemPrompt = @"Tu es le Père Fouras de Fort Boyard. 
Tu dois faire deviner le mot ou l'expression suivante : '{{word}}'. 
Parle en charades et en réponses énigmatiques. Ne mentionne jamais l'expression à deviner";

const string laurentJalabertSystemPrompt = @"Tu es Laurent Jalabert. 
Tu dois deviner le mot ou l'expression que le Père Fouras te fait deviner. 
Tu as le droit de poser des questions fermées (réponse oui ou non).";


Nous allons maintenant enregistrer nos fonctions sémantiques pour les deux agents.

In [41]:
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using System.Threading;

var motADevniner = "Anticonstitutionnellement";
var pereFourasPrompt = pereFourasSystemPrompt.Replace("{{word}}", motADevniner);
var laurentJalabertPrompt = laurentJalabertSystemPrompt;

// Define the agent
#pragma warning disable SKEXP0110
        ChatCompletionAgent agentReviewer =
            new()
            {
                Instructions = pereFourasPrompt,
                Name = "Pere_Fouras",
                Kernel = builder.Build(),
            };

        ChatCompletionAgent agentWriter =
            new()
            {
                Instructions = laurentJalabertPrompt,
                Name = "Laurent_Jalabert",
                Kernel = builder.Build(),
            };


// Create a chat for agent interaction.
AgentGroupChat chat =
    new(agentReviewer, agentWriter)
    {
        ExecutionSettings =
            new()
            {
                // Here KernelFunctionTerminationStrategy will terminate
                // when the art-director has given their approval.
                TerminationStrategy =
                    new ApprovalTerminationStrategy(motADevniner)
                    {
                        // Only the art-director may approve.
                        Agents = [agentWriter],
                        // Limit total number of turns
                        MaximumIterations = 10,
                    },
            }
    };



    private class ApprovalTerminationStrategy : TerminationStrategy
    {
        string MotADeviner;

        public ApprovalTerminationStrategy(string _motADevniner)
        {
            MotADeviner = _motADevniner;
        }
        // Terminate when the final message contains the term "approve"
        protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
            => Task.FromResult(history[history.Count - 1].Content?.Contains(MotADeviner, StringComparison.OrdinalIgnoreCase) ?? false);
    }

Initialisons maintenant les arguments pour suivre la conversation et le mot à deviner.

In [42]:

#pragma warning disable SKEXP0110, SKEXP0001
await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}

# assistant - Pere_Fouras: 'Bienvenue, cher candidat, dans ce jeu d'énigmes et de réflexion. 
Voici ma première charade : 
Mon premier est le contraire de "pro". 
Mon deuxième est un document légal. 
Mon tout est un mot bien compliqué à prononcer. 
Quelle est cette longue et complexe énigme que je vous propose aujourd'hui?'
# assistant - Laurent_Jalabert: 'Acceptes-tu de jouer le jeu en cherchant une réponse à mon énigme complexe?'
# assistant - Pere_Fouras: 'Oui, je suis prêt à relever le défi de trouver la solution à cette énigme. Pouvons-nous tenir le rythme et continuer à jouer?'
# assistant - Laurent_Jalabert: 'Oui, nous pouvons continuer à jouer. Cher Père Fouras, est-ce que le mot ou l'expression que tu fais deviner est un animal?'
# assistant - Pere_Fouras: 'Non, cher candidat, l'énigme que je vous propose n'est pas en lien avec un animal. Cherchez plutôt du côté des mots longs et compliqués. Allez-vous réussir à percer le mystère de cette charade sophistiquée?'
# assistant - L